In [ ]:
import os
import json
from mappings import classToAnsible # need to get the mapping to ansible from json
import yaml
import pprint
from isConfigurableMap import isConfigurableMap # need to get the 
from argument_mapping import argumentMap
from collections import Counter

In [ ]:
Counter(classToAnsible.keys())

In [ ]:
# load the json file

with open('tn-nils-2.json', 'r') as file:
    data = json.load(file)

In [ ]:
# function to map to ansible according to mappings.py

def map_json_to_ansible(json_data, key, map):
    try:
        new_key = map[key]
        json_data[new_key] = json_data.pop(key) # replace with ansible term
    except(KeyError): # no match found, skip
        pass

In [ ]:
def remove_isNotConfigurable(key, delete_key_list):
    try:
        if isConfigurableMap[key] == False:
            delete_key_list.append(key)
        else:
            pass
    except(KeyError):
        pass

In [ ]:
# this is mostly chatGPT, will need to rewrite it and figure out how to do this without it

names = []

def prune_dict(data):
    def contains_name_key(d):
        # Check if 'name' key is in the current dictionary
        if 'name' in d:
            return True
        # Check nested dictionaries
        for key, value in d.items():
            if isinstance(value, dict) and contains_name_key(value):
                return True
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, dict) and contains_name_key(item):
                        return True
        return False

    def prune(d):
        keys_to_delete = []
        for key, value in d.items():
            if isinstance(value, dict):
                if not contains_name_key(value):
                    keys_to_delete.append(key)
                else:
                    prune(value)
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, dict):
                        prune(item)

        for key in keys_to_delete:
            del d[key]

    if isinstance(data, dict):
        prune(data)
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, dict):
                prune(item)

In [ ]:
def reconstruct_yml(data, out_dir=None):
    save_path = "ansible_reconstructed.yml" if out_dir is None else os.path.join(out_dir, "ansible_reconstructed.yml")

    # here define default arguments (to delete)
    default_args = ['', ""]

    # process data recursively
    def process_data(data):

        if isinstance(data, dict):
            keys_to_delete = [] # need this otherwise we get runtime error (dictionary changed size etc)

            for key in data.keys():

                # if value is in default arguments, we can delete it
                if data[key] in default_args:
                    keys_to_delete.append(key)

                else:
                    # nested dictionaries
                    if isinstance(data[key], (dict, list)):
                        try:
                            process_data(data[key])
                        except (KeyError, TypeError) as e:
                            print(f"Error processing key {key}: {e}")

                # uncomment this if we want to delete all non-configurable modules
                remove_isNotConfigurable(key, keys_to_delete)

            # keys to delete
            for key in keys_to_delete:
                del data[key]

            # use the mapping function
            for key in list(data.keys()):
                map_json_to_ansible(data, key, classToAnsible)

        elif isinstance(data, list):
            for item in data:
                if isinstance(item, (dict, list)):
                    process_data(item)

    # Process the initial data
    process_data(data)
    
    # prune_dict(data)

    # Save the resulting dictionary to a YAML file
    with open(save_path, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)

    print(f"YAML file has been saved to {save_path}")

In [ ]:
reconstructed = reconstruct_yml(data)

In [ ]:
def map_isConfigurable_to_ansible(data):
    for key, value in data.items():
        map_json_to_ansible(data, key)

In [ ]:
with open('aci-meta-5.3-2c.json', 'r') as file:
    meta = json.load(file)

In [ ]:
config_map = {} # save configurable map

def configurable_map(data):
    global config_map
    
    def process_data(data, parent_key = None):
        if isinstance(data, dict):
            for key, value in data.items():
                if key == "isConfigurable":

                    # save to config map isConfigurable
                    config_map[parent_key] = value

                # map_json_to_ansible(data, key)

                # pass in key to parent_key to "save" it
                if isinstance(value, dict):
                    process_data(value, parent_key = key)

                elif isinstance(value, list):
                    for item in value:
                        if isinstance(item, dict):
                            process_data(item, parent_key = key)

        if isinstance(data, list):
            for item in data: # iterate over all arrays in the dictionary
                if isinstance(value, dict):
                    process_data(value, parent_key = key)
                    
    process_data(data)

    # map to ansible
    map_isConfigurable_to_ansible(data)
    
    file_name = 'isConfigurableMap.py'

    # Write the dictionary to the file
    with open(file_name, 'w') as file:
        file.write("isConfigurableMap = ")
        pprint.pprint(config_map, stream=file)

    print(f"Dictionary has been saved to {file_name}")

In [ ]:
configurable_map(meta)

In [ ]:
# Save the resulting dictionary to a YAML file
save_path = "ansible_reconstructed.yml"

with open(save_path, 'w') as file:
    yaml.dump(final, file, default_flow_style=False)

print(f"YAML file has been saved to {save_path}")

In [ ]:
clean_playbook = {}

def reorder_playbook(data):
    if isinstance(data, dict):
        for key, value in data.items():
            